## Tutorial 6: Numpy
Objective: Get yourself familiar with Python basic elements.

Code adapted from :
- Nicolas P. Rougier, . (2015, August 21). numpy-tutorial: Version 1.0 (Version 1.0). Zenodo. http://doi.org/10.5281/zenodo.28817 (https://github.com/rougier/numpy-tutorial All code and material is licensed under a Creative Commons Attribution-ShareAlike 4.0.)
- Nicolas P. Rougier, ., Jessica B. Hamrick, ., ibah, ., Gavin Heverly-Coulson, ., Dapid, ., Christoph Deil, ., & Bartosz Telenczuk, . (2016, August 27). numpy-100: Version 1.1 (Version 1.1). Zenodo. http://doi.org/10.5281/zenodo.61020 (https://github.com/rougier/numpy-100 All code and material is licensed under MIT license)
- https://github.com/ksopyla/numpy-tutorial/ All code and material is licensed under MIT license. Copyright (c) 2017 Krzysztof Sopyła

MIT License:
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

Resources:
- https://numpy.org/doc/stable/user/
- https://www.labri.fr/perso/nrougier/from-python-to-numpy/

# Intro

NumPy is the fundamental package for scientific computing with Python. It contains among other things:

* a powerful N-dimensional array object
* sophisticated (broadcasting) functions
* tools for integrating C/C++ and Fortran code
* useful linear algebra, Fourier transform, and random number capabilities

Besides its obvious scientific uses, NumPy can also be used as an efficient multi-dimensional container of generic data. Arbitrary data-types can be defined and this allows NumPy to seamlessly and speedily integrate with a wide variety of projects.


# Compare python list vs numpy array 
We are going to explore numpy through a simple example, implementing the Game of Life. Watch this video for introduction

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('https://www.youtube.com/w atch?v=ouipbDkwHWA')

The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, live or dead. Every cell interacts with its eight neighbours, which are the cells that are directly horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

* Any live cell with fewer than two live neighbours dies, as if by needs caused by underpopulation.
* Any live cell with more than three live neighbours dies, as if by overcrowding.
* Any live cell with two or three live neighbours lives, unchanged, to the next generation.
* Any dead cell with exactly three live neighbours becomes a live cell.

Now you will see two versions of implementations of these: using only Python vs. using Numpy library

## python version 

In [ ]:
def zero_pad(Z): 
    newZ = []
    for r in Z:
        r = [0] + r
        r.append(0)
        newZ.append(r)
    newZ.insert(0, [0]*len(newZ[0]))
    newZ.insert(-1, [0]*len(newZ[0]))
    return newZ

def compute_neighbours(Z):
    shape = len(Z), len(Z[0])
    N  = [[0,]*(shape[0])  for i in range(shape[1])]
    for x in range(1,shape[0]-1):
        for y in range(1,shape[1]-1):
            N[x][y] = Z[x-1][y-1]+Z[x][y-1]+Z[x+1][y-1] \
                    + Z[x-1][y]            +Z[x+1][y]   \
                    + Z[x-1][y+1]+Z[x][y+1]+Z[x+1][y+1]
    return N

def show(Z):
    for l in Z[1:-1]:
        print(l[1:-1])
    print()

def iterate(Z):
    shape = len(Z), len(Z[0])
    N = compute_neighbours(Z)
    for x in range(1,shape[0]-1):
        for y in range(1,shape[1]-1):
            if Z[x][y] == 1 and (N[x][y] < 2 or N[x][y] > 3):
                Z[x][y] = 0
            elif Z[x][y] == 0 and N[x][y] == 3:
                Z[x][y] = 1
    return Z

n_iter = 4 

Z = [[0,0,1,0],
     [1,0,1,0],
     [0,1,1,0],
     [0,0,0,0]]

Z = zero_pad(Z)
print('====== step:{} =======\n'.format(0))
show(Z)

for i in range(n_iter): 
    Z = iterate(Z)
    print('====== step:{} =======\n'.format(i+1))
    show(Z)

====== step:0 =======

[0, 0, 1, 0]
[1, 0, 1, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]

====== step:1 =======

[0, 1, 0, 0]
[0, 0, 1, 1]
[0, 1, 1, 0]
[0, 0, 0, 0]

====== step:2 =======

[0, 0, 1, 0]
[0, 0, 0, 1]
[0, 1, 1, 1]
[0, 0, 0, 0]

====== step:3 =======

[0, 0, 0, 0]
[0, 1, 0, 1]
[0, 0, 1, 1]
[0, 0, 1, 0]

====== step:4 =======

[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 1, 0, 1]
[0, 0, 1, 1]



## numpy version

In [ ]:
import numpy as np

def zero_pad(Z): 
    newZ = np.pad(Z, ((1,1),(1,1)), 'constant')
    return newZ

def compute_neighbours(Z):
    shape = Z.shape
    N  = np.zeros_like(Z)
    kernel = np.ones((3,3))
    kernel[1,1] = 0
    for x in range(1,shape[0]-1):
        for y in range(1,shape[1]-1):
            N[x][y] = np.sum(Z[x-1:x+2, y-1:y+2]*kernel)
    return N
    
def iterate(Z):
    # Count neighbours
    N = compute_neighbours(Z)
    
    # Apply rules
    R1 = (N<2) & (Z==1)
    R2 = (N>3) & (Z==1)
    R3 = ((N==2) | (N==3)) & (Z==1)
    R4 = (N==3) & (Z==0)
    
    Z[R1|R2] = 0
    Z[R3|R4] = 1

    return Z

n_iter = 4 

Z =np.array([[0,0,1,0],
             [1,0,1,0],
             [0,1,1,0],
             [0,0,0,0]])


Z = zero_pad(Z)
print('====== step:{} =======\n'.format(0))
print(Z[1:-1,1:-1])

for i in range(n_iter): 
    Z = iterate(Z)
    print('====== step:{} =======\n'.format(i+1))
    print(Z[1:-1,1:-1])

====== step:0 =======

[[0 0 1 0]
 [1 0 1 0]
 [0 1 1 0]
 [0 0 0 0]]
====== step:1 =======

[[0 1 0 0]
 [0 0 1 1]
 [0 1 1 0]
 [0 0 0 0]]
====== step:2 =======

[[0 0 1 0]
 [0 0 0 1]
 [0 1 1 1]
 [0 0 0 0]]
====== step:3 =======

[[0 0 0 0]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 1 0]]
====== step:4 =======

[[0 0 0 0]
 [0 0 0 1]
 [0 1 0 1]
 [0 0 1 1]]


# 1. creating array

In [ ]:
# import numpy as np
import numpy as np

In [ ]:
# create 1D array
a = np.array([1, 2, 3])  # Create a dimension 1 array
print(type(a))          # Prints "<type 'numpy.ndarray'>"
print(a.shape)            # Prints "(3,)" b/c it is a vector of 3 elements
print(a[0], a[1], a[2])   # Prints "1 2 3"
a[0] = 5                 # Change an element of the array
print(a)                  # Prints "[5, 2, 3]"

<class 'numpy.ndarray'>
(3,)
1 2 3
[5 2 3]


In [ ]:
# create 2D array
b = np.array([[1,2,3],[4,5,6]])   # Create a 2X3 array
print(b.shape)                     # Prints "(2, 3)"
print(b[0, 0], b[0, 1], b[1, 0])  # Prints "1 2 4"


a = np.zeros((2,2))  # Create an array of all zeros
print(a)              # Prints "[[ 0.  0.]
                     #          [ 0.  0.]]"
    
b = np.ones((1,2))   # Create an array of all ones
print(b)              # Prints "[[ 1.  1.]]"


d = np.eye(2)        # Create a 2x2 identity matrix
print(d)              # Prints "[[ 1.  0.]
                     #          [ 0.  1.]]"
    
e = np.random.random((2,2)) # Create an array filled with random values
print (e)                     # Might print "[[ 0.91940167  0.08143941]
                            #               [ 0.68744134  0.87236687]]"

f = np.arange(start=1, stop= 16, step=1).reshape(3, 5) # Create an array filled with range values
# Start and stop indicate range values
# Step indicates the numerical distance between values.
# the code above is the same as np.arange(15).reshape(3, 5)
print(f)


(2, 3)
1 2 4
[[0. 0.]
 [0. 0.]]
[[1. 1.]]
[[1. 0.]
 [0. 1.]]
[[0.741488   0.4100308 ]
 [0.06794146 0.40402002]]
[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]


In [ ]:
# array datatype
x = np.array([1, 2])  # Let numpy choose the datatype
print (x.dtype)         # Prints "int64"

x = np.array([1.0, 2.0])  # Let numpy choose the datatype
print (x.dtype)             # Prints "float64"

x = np.array([1, 2], dtype=np.int64)  # Force a particular datatype
print (x.dtype)       

int64
float64
int64


## 2. indexing array

In [ ]:
# Create the following rank 2 array with shape (3, 4)
# [[ 1  2  3  4]
#  [ 5  6  7  8]
#  [ 9 10 11 12]]
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print (a)
print (a[0, 1])   # Prints "2"

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
2


In [ ]:
# Use slicing to pull out the subarray consisting of the first 2 rows
# and columns 1 and 2; b is the following array of shape (2, 2):
# [[2 3]
#  [6 7]]
b = a[:2, 1:3]
print (b)

[[2 3]
 [6 7]]


In [ ]:
# A slice of an array is a view into the same data, so modifying it will modify the original array.
print (a[0, 1])   # Prints "2"
a[0,1] = 77       # Change 2 to 77
print (a[0, 1])   # Prints "77"

2
77


In [ ]:
# Remember, b refers to a. Changing b will change a too. 
print (a[0, 1])
b[0, 0] = 2      # Change 77 to 2; b[0, 0] is the same piece of data as a[0, 1]
print (a[0, 1])  # Now changes back to 2

77
2


In [ ]:
# If you don't want this, please copy your array!
print (a[0, 1])
c = a[:2, 1:3].copy()
c[0,0] =77 # this only changes c, not a 
print(a[0,1]) 


2
2


In [ ]:
# Create the following rank 2 array with shape (3, 4)
# [[ 1  2  3  4]
#  [ 5  6  7  8]
#  [ 9 10 11 12]]
a = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])

# Two ways of accessing the data in the middle row of the array.
# Mixing integer indexing with slices yields an array of lower rank,
# while using only slices yields an array of the same rank as the
# original array:
row_r1 = a[1, :]    # Rank 1 view of the second row of a  
row_r2 = a[1:2, :]  # Rank 2 view of the second row of a
print (row_r1, row_r1.shape)  # Prints "[5 6 7 8] (4,)"
print (row_r2, row_r2.shape)  # Prints "[[5 6 7 8]] (1, 4)"

# We can make the same distinction when accessing columns of an array:
col_r1 = a[:, 1]
col_r2 = a[:, 1:2]
print (col_r1, col_r1.shape)  # Prints "[ 2  6 10] (3,)"
print (col_r2, col_r2.shape)  # Prints "[[ 2]
                            #          [ 6]
                            #          [10]] (3, 1)"
                            
                            

[5 6 7 8] (4,)
[[5 6 7 8]] (1, 4)
[ 2  6 10] (3,)
[[ 2]
 [ 6]
 [10]] (3, 1)


In [ ]:
#Integer array indexing: 
a = np.array([[1,2], [3, 4], [5, 6]])
print(a)

# An example of integer array indexing.
# The returned array will have shape (3,) and 
print (a[[0, 1, 2], [0, 1, 0]])  # Prints "[1 4 5]"

# The above example of integer array indexing is equivalent to this:
print (np.array([a[0, 0], a[1, 1], a[2, 0]]))  # Prints "[1 4 5]"

# When using integer array indexing, you can reuse the same
# element from the source array:
print (a[[0, 0], [1, 1]])  # Prints "[2 2]"

# Equivalent to the previous integer array indexing example
print (np.array([a[0, 1], a[0, 1]]))  # Prints "[2 2]"        

[[1 2]
 [3 4]
 [5 6]]
[1 4 5]
[1 4 5]
[2 2]
[2 2]


In [ ]:
# Create a new array from which we will select elements
a = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])

print(a)  # prints "array([[ 1,  2,  3],
         #                [ 4,  5,  6],
         #                [ 7,  8,  9],
         #                [10, 11, 12]])"

# Create an array of indices
b = np.array([0, 2, 0, 1])

# Select one element from each row of a using the indices in b
print(a[np.arange(4), b])  # Prints "[ 1  6  7 11]"

# Mutate one element from each row of a using the indices in b
a[np.arange(4), b] += 10

print(a)  # prints "array([[11,  2,  3],
         #                [ 4,  5, 16],
         #                [17,  8,  9],
         #                [10, 21, 12]])
         
         
 

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[ 1  6  7 11]
[[11  2  3]
 [ 4  5 16]
 [17  8  9]
 [10 21 12]]


In [ ]:
#Boolean array indexing         
a = np.array([[1,2], [3, 4], [5, 6]])

bool_idx = (a > 2)  # Find the elements of a that are bigger than 2;
                    # this returns a numpy array of Booleans of the same
                    # shape as a, where each slot of bool_idx tells
                    # whether that element of a is > 2.

print (bool_idx)      # Prints "[[False False]
                    #          [ True  True]
                    #          [ True  True]]"

# We use boolean array indexing to construct a rank 1 array
# consisting of the elements of a corresponding to the True values
# of bool_idx
print (a[bool_idx])  # Prints "[3 4 5 6]"

# We can do all of the above in a single concise statement:
print (a[a > 2])     # Prints "[3 4 5 6]"

[[False False]
 [ True  True]
 [ True  True]]
[3 4 5 6]
[3 4 5 6]


## 3. shaping & modifying matrix

In [ ]:
# combining array
a = np.array([[1,5,9], [2,6,10]])
b = np.array([[3,7,11], [4,8,12]])

# Stack two arrays row-wise
print(np.vstack((a,b)))

# vstack is the same as concatenate arrays along the axis 0 (row)
print(np.concatenate((a,b), axis=0))

# Stack two arrays column-wise
print(np.hstack((a,b)))

# hstack is the same as concatenate arrays along the axis 1 (column)
print(np.concatenate((a,b), axis=1))

[[ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]
 [ 4  8 12]]
[[ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]
 [ 4  8 12]]
[[ 1  5  9  3  7 11]
 [ 2  6 10  4  8 12]]
[[ 1  5  9  3  7 11]
 [ 2  6 10  4  8 12]]


In [ ]:
# flatten
a = np.array([[1,2], [3,4]])
a = a.flatten()
print(a)

[1 2 3 4]


In [ ]:
# reshape
a = np.array([[1,2], [3,4]])
a = a.reshape((4,)) # make it as vector
print(a)

a = np.array([1,5,9, 2,6,10]) #originally vector
a = a.reshape((3,2)) # make it 3*2
print(a)

[1 2 3 4]
[[ 1  5]
 [ 9  2]
 [ 6 10]]


In [ ]:
# add new axis
x = np.array([1, 2]) # shape (2,)
print(x)
y = np.expand_dims(x, axis=0) # add new axis row-wise (shape becomes 1*2) =  x[np.newaxis, :]
print(y)
z = np.expand_dims(x, axis=1) # add new axis column-wise (shape becomes 2*1)  x[:, np.newaxis]
print(z)

[1 2]
[[1 2]]
[[1]
 [2]]


## 4. matrix computation

In [ ]:
#basic math operation
x = np.array([[1,2],[3,4]])
y = np.array([[5,6],[7,8]])

# Elementwise sum; both produce the array
# [[ 6.0  8.0]
#  [10.0 12.0]]
print (x + y)
print (np.add(x, y))

# Elementwise difference; both produce the array
# [[-4.0 -4.0]
#  [-4.0 -4.0]]
print (x - y)
print (np.subtract(x, y))

# Elementwise product; both produce the array
# [[ 5.0 12.0]
#  [21.0 32.0]]
print (x * y)
print (np.multiply(x, y))

# Elementwise division; both produce the array
# [[ 0.2         0.33333333]
#  [ 0.42857143  0.5       ]]
print (x / y)
print (np.divide(x, y))

# Elementwise square root; produces the array
# [[ 1.          1.41421356]
#  [ 1.73205081  2.        ]]
print (np.sqrt(x))

# Summation
print (np.sum(x))  # Compute sum of all elements; prints "10"
print (np.sum(x, axis=0))  # Compute sum of each column; prints "[4 6]"
print (np.sum(x, axis=1))  # Compute sum of each row; prints "[3 7]"

[[ 6  8]
 [10 12]]
[[ 6  8]
 [10 12]]
[[-4 -4]
 [-4 -4]]
[[-4 -4]
 [-4 -4]]
[[ 5 12]
 [21 32]]
[[ 5 12]
 [21 32]]
[[0.2        0.33333333]
 [0.42857143 0.5       ]]
[[0.2        0.33333333]
 [0.42857143 0.5       ]]
[[1.         1.41421356]
 [1.73205081 2.        ]]
10
[4 6]
[3 7]


In [ ]:
#vector matrix operation
v = np.array([9,10])
w = np.array([11, 12])

x = np.array([[1,2],[3,4]])
# this is
# [[1,2]
# [3,4]]

y = np.array([[5,6],[7,8]])
# this is
# [[5,6]
# [7,8]]

# Inner product of vectors; both produce 219
print (v.dot(w))
print (np.dot(v, w))

# Matrix / vector product; both produce the rank 1 array [29 67]
print (x.dot(v))
print (np.dot(x, v))

# Matrix / matrix product; both produce the rank 2 array = matrix multiplication 
# but using np.matmul or a @ b is preferred.
# [[19 22]
#  [43 50]]
print (x.dot(y))
print (np.dot(x, y))

# cf. elementwise mutiplication 
print(x*y)


219
219
[29 67]
[29 67]
[[19 22]
 [43 50]]
[[19 22]
 [43 50]]
[[ 5 12]
 [21 32]]


In [ ]:
# Transpose
x = np.array([[1,2], [3,4]])
print (x)    # Prints "[[1 2]
           #          [3 4]]"
print (x.T)  # Prints "[[1 3]
           #          [2 4]]"

# Note that taking the transpose of a rank 1 array does nothing:
v = np.array([1,2,3])
print (v)    # Prints "[1 2 3]"
print (v.T)  # Prints "[1 2 3]"

[[1 2]
 [3 4]]
[[1 3]
 [2 4]]
[1 2 3]
[1 2 3]


## 5. Array broadcasting

You want to add the vector v = [1,0,1] to each row of the matrix x and save to y

In [ ]:
x = np.array([[1,2,3], [4,5,6], [7,8,9]])
v = np.array([1, 0, 1])

There can be several ways to to do this...

In [ ]:
#1. solve by using array indexing
y = np.empty_like(x)   # Create an empty matrix with the same shape as x

# Add the vector v to each row of the matrix x with an explicit loop
for i in range(3):
    y[i, :] = x[i, :] + v

print (y)

[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]]


In [ ]:
#2. solve by np.tile
vv = np.tile(v, (3, 1))  # Stack v in a shape of (4,1); 4 copies of v on top of each other
print (vv)                 # Prints "[[1 0 1]
                         #          [1 0 1]
                         #          [1 0 1]

y = x + vv  # Add x and vv elementwise
print (y)

[[1 0 1]
 [1 0 1]
 [1 0 1]]
[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]]


But this is much *simpler* way! Use array broadcasting

In [ ]:
#2. solve by broadcasting
y = x + v  # Add v to each row of x using broadcasting
print (y)  

[[ 2  2  4]
 [ 5  5  7]
 [ 8  8 10]]


In [ ]:
# if you want to add a vector to each column of a matrix, not row
# x has shape (2, 3) and w has shape (2,).
# If we transpose x then it has shape (3, 2) and can be broadcast
# against w to yield a result of shape (3, 2); transposing this result
# yields the final result of shape (2, 3) which is the matrix x with
# the vector w added to each column. Gives the following matrix:
# [[ 5  6  7]
#  [ 9 10 11]]
print ((x.T + v).T)

[[ 2  3  4]
 [ 4  5  6]
 [ 8  9 10]]


In [ ]:
# Multiply a matrix by a constant:
# x has shape (2, 3). Numpy treats scalars as arrays of shape ();
# these can be broadcast together to shape (2, 3), producing the
# following array:
# [[ 2  4  6]
#  [ 8 10 12]]
print (x * 2)

[[ 2  4  6]
 [ 8 10 12]
 [14 16 18]]


# 6. Miscellenous

Generate random numbers

In [ ]:
# generate random integer
print(np.random.randint(10)) #output a random integer between 0 and 10
# generate random floats
print(np.random.rand()) # output a random float between 0 and 1
# generate a random value based on an array of values.
print(np.random.choice([3, 5, 7, 9]))

# generate random integer in matrix
print(np.random.randint(10, size=(2,2))) #output a random integer between 0 and 10  in 2x2 matrix
# generate random floats  in matrix
print(np.random.rand(2,2)) # output a random float between 0 and 1  in 2x2 matrix
# generate a random value based on an array of values in matrix
print(np.random.choice([3, 5, 7, 9], size=(2,2)))

4
0.49316548738802857
7
[[9 1]
 [2 4]]
[[0.26963751 0.71754773]
 [0.74143828 0.99573257]]
[[7 5]
 [9 3]]


Search arrays

In [ ]:
#You can search an array for a certain value, and return the indexes that get a match.
arr = np.array([1, 2, 3, 4, 5, 4, 4])
x = np.where(arr == 4)
print(x)

(array([3, 5, 6]),)


# Numpy Exercises

## Q1. Create a null vector of size (3,3) but the value at (first row, second column) as 5
hint: use np.zeros 

## Q2. Create a 10x10 array with random values and find the minimum/maximum/mean values

hint: .min(), .max(), .mean()

## Q3. Run the code below and discuss the results 

In [ ]:
print(0 * np.nan)
print(np.nan == np.nan)
print(np.inf > np.nan)
print(np.nan - np.nan)
print(np.nan in set([np.nan]))
print(0.3 == 3 * 0.1)

nan
False
False
nan
True
False


Discussion:

# Q4. Normalize a 5x5 random matrix 

hint: (x -mean)/std

# Q5. Multiply a the following 5x3 matrix by a 3x2 matrix
hint: @ or np.matmul

In [1]:
a = np.arange(15).reshape(5,3)
b = np.arange(6).reshape(3,2)
## matrix multiplaction
c = 
print(c)

SyntaxError: ignored

# Q6. Create random vector of size 10 and replace the maximum value by 0
hint: np.argmax

# Q7. Find the closest value 

hint: get the absolute difference between Z and v using `np.abs`, and find the location of the smallest difference using `.argmin()`

In [2]:
v = 3.2
Z = np.arange(100)

# find the value that is closest to 3.2 in vector Z
index = 
print(Z[index])

SyntaxError: ignored

## Q8. Find the indexes where the values are even

hint: np.where

In [3]:
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8])
# your implementation
x = 
print(x) # it should print [1,3,5,7]

SyntaxError: ignored